In [304]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm
import time

# using raw web scrapping to generate structured data from the companiesmarketcap webpage:





My goal was to get the 10 largest companies from each of the ten richest countries in the world.

Source:  `https://companiesmarketcap.com/`


 ## Generating a data frame with the largest companies from each country :


In [98]:
stocks_country = pd.DataFrame(columns = ['Rank', 'Name', 'Market Cap', 'Price', 'Today', 'Price (30 days)', 'Country'] )
stocks_country

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country


In [182]:
lista_1 = ('usa','china','japan','germany','india','united-kingdom','france','brazil','italy','canada')
lista_2 = ('the-usa','china','japan','germany','india','the-uk','france','brazil','italy','canada')

In [100]:
for i in list(range(0,len(lista_1))):
    url = f'https://companiesmarketcap.com/{lista_1[i]}/largest-companies-in-{lista_2[i]}-by-market-cap/'
    print(url)
    response = requests.get(url)
    html = response.content
    stocks_country = stocks_country.append(pd.read_html(html)[0].head(10))

https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
https://companiesmarketcap.com/china/largest-companies-in-china-by-market-cap/
https://companiesmarketcap.com/japan/largest-companies-in-japan-by-market-cap/
https://companiesmarketcap.com/germany/largest-companies-in-germany-by-market-cap/
https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
https://companiesmarketcap.com/united-kingdom/largest-companies-in-the-uk-by-market-cap/
https://companiesmarketcap.com/france/largest-companies-in-france-by-market-cap/
https://companiesmarketcap.com/brazil/largest-companies-in-brazil-by-market-cap/
https://companiesmarketcap.com/italy/largest-companies-in-italy-by-market-cap/
https://companiesmarketcap.com/canada/largest-companies-in-canada-by-market-cap/


In [233]:
 stocks_country = stocks_country.reset_index()

## Adding a new column with the symbols:

Each company that trades in the stock market as a symbol that represents it. If you notice, in the  `Name` column, were we should only have the name, we have a sequence of letters and numbers, those are the Tickers or Symbols.

We will need those symbols later on to access Alphavantage's API in order to download the daily historic prices of those companies. 

I decided to access the web page again, make a list containing the tickers and include it to the data frame we generated above.



In [195]:

ticker_list = []
lista_intermedia = []

for i in list(range(0,len(lista_1))):
    url = f'https://companiesmarketcap.com/{lista_1[i]}/largest-companies-in-{lista_2[i]}-by-market-cap/'
    print(url)
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    for ticker in  soup.find_all('div', attrs = {'class':'company-code'}):
        lista_intermedia.append(ticker.text)
        lista_intermedia = lista_intermedia[ : 10]
    for  ticker in lista_intermedia:
        ticker_list.append(ticker)
    lista_intermedia =[]


https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
https://companiesmarketcap.com/china/largest-companies-in-china-by-market-cap/
https://companiesmarketcap.com/japan/largest-companies-in-japan-by-market-cap/
https://companiesmarketcap.com/germany/largest-companies-in-germany-by-market-cap/
https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
https://companiesmarketcap.com/united-kingdom/largest-companies-in-the-uk-by-market-cap/
https://companiesmarketcap.com/france/largest-companies-in-france-by-market-cap/
https://companiesmarketcap.com/brazil/largest-companies-in-brazil-by-market-cap/
https://companiesmarketcap.com/italy/largest-companies-in-italy-by-market-cap/
https://companiesmarketcap.com/canada/largest-companies-in-canada-by-market-cap/


In [234]:
stocks_country['ticker'] = ticker_list
#stocks_country.head()
stocks_country.tail()

In [256]:
stocks_country = stocks_country.drop('index', axis = 1)

 # Using Alphavantage API to get the historical stock prices of the last 10 years of those companies:
 
 What we want to do now is to use the tickers we got from `companiesmarketcap` and use them to download the daily adjusted prices from Alphavantage API
 
 
 
 The first thing we need to do is:
 
 * Checkout if the tickers are right
 * Separate the right ones and the wrong ones
 * Download the right ones and with the wrong, try to correct them.
 
 
 

In [240]:
from dotenv import  load_dotenv
load_dotenv( 'password.env')

import os

ALPHAVANTAGE_KEY = os.getenv('senha')

In [242]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key = ALPHAVANTAGE_KEY, output_format='pandas')


## Finding out the wrong tickers:

Many companies have different tickers depending on the sock exchange or market we are looking.

We need to make sure we have the symbols we have work on Alphavantage's API.

To do that, we will use the function `get_symbol_search` to see if they exist. 

**NOTE:** Aphavantage's APi allows us only to search for 5 tickers per minute. 




In [349]:

right_ticker = []
x = 0
for ticker in ticker_list:
    x += 1
    print(ticker)
    print(x)
    if (x % 5 == 0):
        time.sleep(60)
    y = ts.get_symbol_search(ticker)[0].empty
    right_ticker.append(y)

#this function we will give us the answer True if the ticker is Wrong and False if the Ticker is right.

AAPL
1
MSFT
2
AMZN
3
GOOG
4
TSLA
5
FB
6
BRK-A
7
V
8
JNJ
9
JPM
10
TCEHY
11
BABA
12
600519.SS
13
MPNGF
14
1398.HK
15
PDD
16
PNGAY
17
3968.HK
18
000858.SZ
19
CICHY
20
TM
21
SFTBF
22
SNE
23
KYCCF
24
NTDMF
25
NPPXF
26
FRCOF
27
4519.T
28
NNDNF
29
RCRRF
30
SAP
31
SIEGY
32
VOW3.DE
33
ALIZF
34
DTE.DE
35
DAI.DE
36
BFFAF
37
MRK.DE
38
ADDDF
39
BAYZF
40
RELIANCE.NS
41
TCS.NS
42
HDB
43
INFY
44
HINDUNILVR.NS
45
HDFC.NS
46
IBN
47
KOTAKBANK.NS
48
SBIN.NS
49
BHARTIARTL.NS
50
LIN
51
AZN
52
HSBC
53
RIO
54
DEO
55
GSK
56
BTI
57
BP
58
LS4C.F
59
RBGPF
60
LVMUY
61
OR.PA
62
SNY
63
HESAF
64
TOT
65
CHDRF
66
KER.PA
67
SBGSF
68
AI.PA
69
ESLOF
70
VALE
71
PBR
72
ITUB
73
ABEV
74
BBD
75
WEGE3.SA
76
BSBR
77
STNE
78
XP
79
B3SA3.SA
80
ENLAY
81
RACE
82
ISNPY
83
E
84
G.MI
85
UCG.MI
86
SNMRF
87
PRDSF
88
MONC.MI
89
ATL.MI
90
SHOP
91
RY
92
TD
93
CNI
94
ENB
95
BNS
96
BAM
97
BMO
98
CP
99
LULU
100


In [350]:
#we are creating a new column inside the data frame to include

stocks_country['right_ticker'] = right_ticker

In [353]:
stocks_country.loc[stocks_country['right_ticker'] == True]

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country,ticker,right_ticker
14,5,ICBC 1398.HK,$266.72 B,$0.62,-0.82%,NaN,🇨🇳 China,1398.HK,True
17,8,CM Bank 3968.HK,$213.91 B,$8.23,0.71%,NaN,🇨🇳 China,3968.HK,True
27,8,Chugai Pharmaceutical 4519.T,$83.46 B,$50.81,-1.42%,NaN,🇯🇵 Japan,4519.T,True
40,1,Reliance Industries RELIANCE.NS,$181.45 B,$26.83,0.24%,NaN,🇮🇳 India,RELIANCE.NS,True
41,2,Tata TCS.NS,$161.20 B,$43.58,-1.16%,NaN,🇮🇳 India,TCS.NS,True
44,5,Hindustan Unilever HINDUNILVR.NS,$71.45 B,$30.40,-0.91%,NaN,🇮🇳 India,HINDUNILVR.NS,True
45,6,Housing Development Finance Corporation HDFC.NS,$67.83 B,$37.68,0.50%,NaN,🇮🇳 India,HDFC.NS,True
47,8,Kotak Mahindra Bank KOTAKBANK.NS,$52.96 B,$26.74,-0.41%,NaN,🇮🇳 India,KOTAKBANK.NS,True
48,9,State Bank of India SBIN.NS,$48.33 B,$5.42,-0.57%,NaN,🇮🇳 India,SBIN.NS,True
49,10,Bharti Airtel BHARTIARTL.NS,$44.82 B,$8.22,0.28%,NaN,🇮🇳 India,BHARTIARTL.NS,True


## generating a Data Frame containing the daily adjusted prices of the Stocks we know we have the right ticker:

In [ ]:

stocks_right_ticker = stocks_country.loc[stocks_country['right_ticker'] == False]
right_stocks = list(stocks_right_ticker['ticker'])


In [355]:

daily_adjusted_prices = pd.DataFrame(columns = ['date','1. open', '2. high', '3. low', '4. close', '5. adjusted close','6. volume', '7. dividend amount', '8. split coefficient'])


In [393]:

x = 0



for ticker in right_stocks:
    x += 1
    print(ticker)
    print(x)
    if (x % 5 == 0):
        time.sleep(60)   
    ts = TimeSeries(key = ALPHAVANTAGE_KEY, output_format='pandas')
    data = ts.get_daily_adjusted(symbol = ticker , outputsize = 'full')[0]
    daily_adjusted_prices = daily_adjusted_prices.append(data)

  

AAPL
1
MSFT
2
AMZN
3
GOOG
4
TSLA
5
FB
6
BRK-A
7
V
8
JNJ
9
JPM
10
TCEHY
11
BABA
12
600519.SS
13
MPNGF
14
PDD
15
PNGAY
16
000858.SZ
17
CICHY
18
TM
19
SFTBF
20
SNE
21
KYCCF
22
NTDMF
23
NPPXF
24
FRCOF
25
NNDNF
26
RCRRF
27
SAP
28
SIEGY
29
VOW3.DE
30
ALIZF
31
DTE.DE
32
DAI.DE
33
BFFAF
34
MRK.DE
35
ADDDF
36
BAYZF
37
HDB
38
INFY
39
IBN
40
LIN
41
AZN
42
HSBC
43
RIO
44
DEO
45
GSK
46
BTI
47
BP
48
LS4C.F
49
RBGPF
50
LVMUY
51
OR.PA
52
SNY
53
HESAF
54
TOT
55
CHDRF
56
KER.PA
57
SBGSF
58
AI.PA
59
ESLOF
60
VALE
61
PBR
62
ITUB
63
ABEV
64
BBD
65
WEGE3.SA
66
BSBR
67
STNE
68
XP
69
B3SA3.SA
70
ENLAY
71
RACE
72
ISNPY
73
E
74
SNMRF
75
PRDSF
76
SHOP
77
RY
78
TD
79
CNI
80
ENB
81
BNS
82
BAM
83
BMO
84
CP
85
LULU
86


In [395]:
daily_adjusted_prices

,date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2021-02-09,NaN,136.62,137.877,135.85,136.01,136.010000,75986989.0,0.000,1.0
2021-02-08,NaN,136.03,136.960,134.92,136.91,136.910000,71297214.0,0.000,1.0
2021-02-05,NaN,137.35,137.420,135.86,136.76,136.760000,75693830.0,0.205,1.0
2021-02-04,NaN,136.30,137.400,134.59,137.39,137.184364,84183061.0,0.000,1.0
2021-02-03,NaN,135.76,135.770,133.61,133.94,133.739528,89880937.0,0.000,1.0
...,...,...,...,...,...,...,...,...,...
2007-08-02,NaN,32.00,33.830,31.98,33.05,16.525000,984671.0,0.000,1.0
2007-08-01,NaN,32.98,33.250,30.40,31.08,15.540000,1530573.0,0.000,1.0
2007-07-31,NaN,32.00,34.170,30.21,32.14,16.070000,2220748.0,0.000,1.0
2007-07-30,NaN,28.86,31.850,28.36,29.98,14.990000,2883501.0,0.000,1.0


## finding out symbols we can use for the ones alphavantage couldn't identify

In [484]:
corrected_tickers = stocks_country.loc[stocks_country['right_ticker'] == True]
corrected_tickers

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country,ticker,right_ticker
14,5,ICBC 1398.HK,$266.72 B,$0.62,-0.82%,NaN,🇨🇳 China,1398.HK,True
17,8,CM Bank 3968.HK,$213.91 B,$8.23,0.71%,NaN,🇨🇳 China,3968.HK,True
27,8,Chugai Pharmaceutical 4519.T,$83.46 B,$50.81,-1.42%,NaN,🇯🇵 Japan,4519.T,True
40,1,Reliance Industries RELIANCE.NS,$181.45 B,$26.83,0.24%,NaN,🇮🇳 India,RELIANCE.NS,True
41,2,Tata TCS.NS,$161.20 B,$43.58,-1.16%,NaN,🇮🇳 India,TCS.NS,True
44,5,Hindustan Unilever HINDUNILVR.NS,$71.45 B,$30.40,-0.91%,NaN,🇮🇳 India,HINDUNILVR.NS,True
45,6,Housing Development Finance Corporation HDFC.NS,$67.83 B,$37.68,0.50%,NaN,🇮🇳 India,HDFC.NS,True
47,8,Kotak Mahindra Bank KOTAKBANK.NS,$52.96 B,$26.74,-0.41%,NaN,🇮🇳 India,KOTAKBANK.NS,True
48,9,State Bank of India SBIN.NS,$48.33 B,$5.42,-0.57%,NaN,🇮🇳 India,SBIN.NS,True
49,10,Bharti Airtel BHARTIARTL.NS,$44.82 B,$8.22,0.28%,NaN,🇮🇳 India,BHARTIARTL.NS,True


In [481]:
#This was done by hand

tickers_corrected =('601398.SHH','600036.SHH','CHGCY','RELIANCE.BSE','TATASTEEL.BSE','500696.BSE','HDFC.BSE','500247.BSE','SBIN.BSE','532454.BSE','ARZGY','UNCRY','MONRY','ATASY')
len(tickers_corrected)

14

In [487]:
#ts.get_symbol_search('Reliance Industries')[0]  

In [485]:
corrected_tickers['ticker'] = tickers_corrected

C:\Users\jmcac\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [486]:
corrected_tickers

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country,ticker,right_ticker
14,5,ICBC 1398.HK,$266.72 B,$0.62,-0.82%,NaN,🇨🇳 China,601398.SHH,True
17,8,CM Bank 3968.HK,$213.91 B,$8.23,0.71%,NaN,🇨🇳 China,600036.SHH,True
27,8,Chugai Pharmaceutical 4519.T,$83.46 B,$50.81,-1.42%,NaN,🇯🇵 Japan,CHGCY,True
40,1,Reliance Industries RELIANCE.NS,$181.45 B,$26.83,0.24%,NaN,🇮🇳 India,RELIANCE.BSE,True
41,2,Tata TCS.NS,$161.20 B,$43.58,-1.16%,NaN,🇮🇳 India,TATASTEEL.BSE,True
44,5,Hindustan Unilever HINDUNILVR.NS,$71.45 B,$30.40,-0.91%,NaN,🇮🇳 India,500696.BSE,True
45,6,Housing Development Finance Corporation HDFC.NS,$67.83 B,$37.68,0.50%,NaN,🇮🇳 India,HDFC.BSE,True
47,8,Kotak Mahindra Bank KOTAKBANK.NS,$52.96 B,$26.74,-0.41%,NaN,🇮🇳 India,500247.BSE,True
48,9,State Bank of India SBIN.NS,$48.33 B,$5.42,-0.57%,NaN,🇮🇳 India,SBIN.BSE,True
49,10,Bharti Airtel BHARTIARTL.NS,$44.82 B,$8.22,0.28%,NaN,🇮🇳 India,532454.BSE,True


## Downloading the historic daily adjusted prices for this stocks and appending them to the second data frame:

In [495]:
x = 0



for ticker in tickers_corrected:
    x += 1
    print(ticker)
    print(x)
    if (x % 5 == 0):
        time.sleep(60)   
    ts = TimeSeries(key = ALPHAVANTAGE_KEY, output_format='pandas')
    data = ts.get_daily_adjusted(symbol = ticker , outputsize = 'full')[0]
    daily_adjusted_prices = daily_adjusted_prices.append(data)


601398.SHH
1
600036.SHH
2
CHGCY
3
RELIANCE.BSE
4
TATASTEEL.BSE
5
500696.BSE
6
HDFC.BSE
7
500247.BSE
8
SBIN.BSE
9
532454.BSE
10
ARZGY
11
UNCRY
12
MONRY
13
ATASY
14


In [500]:
daily_adjusted_prices.to_csv('daily_adjusted_prices.csv', index=False)

## substituting the tickers in the first data frame:

In [489]:
stocks_country = stocks_right_ticker

In [491]:
stocks_country = stocks_country.append(corrected_tickers)

In [499]:
stocks_country.to_csv('stocks_country.csv', index=False)